# **KNN (K-Nearest Neighbors) — Módulo 3, Notebook 1/6**

---

## Índice

1. [Introdução ao KNN](#introducao)
2. [Como o KNN Funciona?](#como-funciona)
3. [Intuição Prática](#intuicao)
4. [Formalização Matemática](#formalizacao)
5. [Implementação do Zero](#from-scratch)
6. [Implementação com Scikit-Learn](#sklearn)
7. [Vantagens e Desvantagens](#vantagens-desvantagens)

---

<a id='introducao'></a>
## **Introdução ao KNN**

Para compreender o funcionamento do KNN, vamos partir de um exemplo intuitivo. Imagine que somos novos usuários na Netflix. A plataforma precisa prever se vamos gostar de um filme específico que nunca assistimos, digamos, "Crepúsculo". Como ela faria isso?

A estratégia é direta: **encontrar pessoas com gostos semelhantes aos nossos e verificar o que elas acharam do filme**.

Se 8 de 10 usuários que apreciaram os mesmos filmes que nós também gostaram de "Crepúsculo", existe uma probabilidade alta de também gostarmos. Esse é precisamente o princípio do KNN — classificar novos dados com base nos exemplos mais "próximos" que já conhece.

O KNN (K-Nearest Neighbors, ou K-Vizinhos Mais Próximos) é um dos algoritmos de machine learning mais simples e intuitivos. Não envolve fórmulas complexas ou treino sofisticado — o modelo memoriza os dados e, quando precisa fazer uma previsão, busca os exemplos mais semelhantes.

<a id='como-funciona'></a>
## **Como o KNN Funciona?**

Retomando os conceitos fundamentais, vamos relembrar que trabalhamos com um **espaço de características**. Cada observação em nossa base de dados é representada por um **vetor de características** neste espaço. 

O algoritmo KNN opera em três etapas fundamentais:

### **Passo 1: Armazenar os Dados**
O algoritmo armazena todos os exemplos de treinamento. Cada observação corresponde a um ponto no espaço de características.

### **Passo 2: Calcular Distâncias**
Quando precisamos classificar uma nova observação, o KNN calcula a distância entre esse novo ponto e **todos** os pontos do conjunto de treino. A métrica mais comum é a **distância euclidiana** (distância em linha reta entre dois pontos).

### **Passo 3: Votar pela Classe**
O algoritmo identifica os K vizinhos mais próximos (daí o "K" no nome) e realiza uma votação: a classe mais frequente entre esses vizinhos é atribuída ao novo ponto.

O gif abaixo ilustra esse processo de maneira visual:

<p align=center>
<img src="https://machinelearningknowledge.ai/wp-content/uploads/2018/08/KNN-Classification.gif" width=600 height=400></img>
</p>

### **Parâmetros Importantes**

Quando trabalhamos com KNN, dois parâmetros são fundamentais:

**1. Número de vizinhos (K):**
- K pequeno (ex: K=1): Modelo mais complexo, sensível a ruído
- K grande (ex: K=50): Modelo mais simples, fronteiras mais suaves

**2. Métrica de distância:**
- **Distância Euclidiana** (mais comum): linha reta entre pontos
- Distância de Manhattan: soma das diferenças absolutas
- Outras métricas para casos específicos

<a id='intuicao'></a>
## **Intuição Prática: Classificando Flores Iris**

Com a compreensão conceitual estabelecida, vamos consolidar esse conhecimento através de um exemplo prático. Utilizaremos o dataset mais clássico de machine learning: o **Iris Dataset**.

### **O Problema**

Considere que encontramos uma flor íris na natureza e medimos suas características:
- Comprimento da pétala: 4.5 cm
- Largura da pétala: 1.5 cm

Sabemos que existem 3 espécies de íris: **Setosa**, **Versicolor** e **Virginica**.

**Desafio:** Determinar a qual espécie essa flor pertence.

Vamos aplicar o KNN para resolver esse problema! Implementaremos o algoritmo passo a passo para compreender exatamente como ele toma essa decisão.

In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# Carregando o Iris Dataset
iris = load_iris()

X = iris.data  # Features: 4 características das flores
y = iris.target  # Labels: 3 classes (0, 1, 2) das flores

print("Classes:")
for i, nome in enumerate(iris.target_names):
    print(f"  Classe {i}: {nome.capitalize()} ({np.sum(y == i)} amostras)")
print("=" * 70)

print("\nFeatures (cm):")
for i, nome in enumerate(iris.feature_names):
    print(f"  {i+1}. {nome}")
print(f"\nFormato dos dados: {X.shape}")
print("=" * 70)

# Visualizando os dados
print("\nVisualizando os dados:")
pd.DataFrame(X, columns=iris.feature_names).head()

In [ ]:
# Simplificando para visualização: utilizaremos apenas 2 features
# Comprimento e largura da pétala (features mais discriminativas)
X_2d = X[:, [2, 3]]

print("Reduzimos o dataset para 2 features para facilitar a visualização:")
print("  - Comprimento da pétala")
print("  - Largura da pétala")
print(f"\nNovo formato: {X_2d.shape}")

In [ ]:
# Criando a flor que queremos classificar
novo_ponto = np.array([4.5, 1.5])  # Comprimento e largura da pétala

print("Nossa flor desconhecida:")
print(f"  Comprimento da pétala: {novo_ponto[0]} cm")
print(f"  Largura da pétala: {novo_ponto[1]} cm")
print(f"  Classe: ???")
print("\nVamos descobrir usando KNN!")

In [ ]:
# ===================================================================
# PASSO 1: CALCULAR DISTÂNCIAS
# ===================================================================

def euclidean_distance(X, point):
    """
    Calcula a distância Euclidiana entre um ponto e todos os pontos em X.
    
    Fórmula: d = sqrt((x2 - x1)^2 + (y2 - y1)^2)
    """
    distance = np.sqrt(np.sum((X - point) ** 2, axis=1))
    return distance

# Calculando as distâncias
distancias = euclidean_distance(X_2d, novo_ponto)

print("Distâncias calculadas:")
for i in range(5):
    print(f"  Distância até ponto {i+1}: {distancias[i]:.4f} cm")
print("...")
print(f"  Distância até ponto 150: {distancias[-1]:.4f} cm")
print(f"\nTotal de distâncias calculadas: {len(distancias)}")

In [ ]:
# ===================================================================
# PASSO 2: ENCONTRAR OS K VIZINHOS MAIS PRÓXIMOS
# ===================================================================

def find_knn(distancias, k):
    """
    Retorna os índices dos k vizinhos mais próximos
    """
    indices_dos_vizinhos = np.argsort(distancias)[:k]
    return indices_dos_vizinhos

k = 7
k_vizinhos = find_knn(distancias, k)

print(f"Os {k} vizinhos mais próximos:")
for i, idx in enumerate(k_vizinhos):
    classe = iris.target_names[y[idx]]
    print(f"  {i+1}º Vizinho: Flor #{idx+1} - Classe: {classe.capitalize()} - Distância: {distancias[idx]:.4f} cm")

print("\nNote: Os vizinhos mais próximos têm as menores distâncias.")

In [ ]:
# ===================================================================
# PASSO 3: FAZER A PREVISÃO DA CLASSE
# ===================================================================

def predict_class(y_train, k_vizinhos_indices):
    """
    Faz a previsão da classe com base nos k vizinhos mais próximos
    """
    k_classes = y_train[k_vizinhos_indices]
    contagem = Counter(k_classes)
    most_common = contagem.most_common(1)
    return most_common[0][0], contagem

classe_prevista, contagem_classes = predict_class(y, k_vizinhos)

print("Votação dos vizinhos:")
for classe_id in sorted(contagem_classes.keys()):
    nome_classe = iris.target_names[int(classe_id)]
    quantidade = contagem_classes[classe_id]
    percentual = (quantidade / k) * 100
    barra = '█' * quantidade
    print(f"  {nome_classe.capitalize():12s}: {quantidade}/{k} ({percentual:5.1f}%) {barra}")

resultado_nome = iris.target_names[int(classe_prevista)]

print(f"\n{'='*70}")
print(f"RESULTADO FINAL: A flor é da classe '{resultado_nome.upper()}'")
print(f"   Confiança: {contagem_classes[classe_prevista]}/{k} ({(contagem_classes[classe_prevista]/k)*100:.1f}%)")
print(f"{'='*70}")

### **Desafio**

Agora que você entendeu os 3 passos do KNN com o exemplo das flores, consegue identificar como o mesmo algoritmo funcionaria com a recomendação da Netflix que mencionamos na introdução?

**Desafio:** 
- Qual seria o "espaço de características"?
- Como seria feita a votação final?
- Qual seria o resultado previsto para você?

Pense sobre isso antes de continuar!

### **Visualizando o Processo**

Vamos agora visualizar graficamente como o KNN tomou essa decisão. A figura a seguir ilustra as três etapas do algoritmo de forma sequencial:

In [ ]:
# Criando visualização dos 3 passos do KNN
np.random.seed(42)

# Criando amostra para o gráfico
amostra_indices = []
for classe in range(3):
    indices_classe = np.where(y == classe)[0]
    amostra_classe = np.random.choice(indices_classe, 10, replace=False)
    amostra_indices.extend(amostra_classe)

X_amostra = X_2d[amostra_indices]
y_amostra = y[amostra_indices]

# Recalculando para a amostra
novo_ponto_viz = np.array([4.5, 1.5])
dist_viz = euclidean_distance(X_amostra, novo_ponto_viz)
k_viz = 5
vizinhos_viz = find_knn(dist_viz, k_viz)
classe_viz, contagem_viz = predict_class(y_amostra, vizinhos_viz)

# Cores e labels
cores = ['r', 'g', 'b']
labels = iris.target_names

fig, axes = plt.subplots(1, 3, figsize=(20, 4))
fig.suptitle("Classificação KNN - Visualização dos 3 Passos do Algoritmo", fontsize=16, fontweight='bold', y=1.02)

# -----------------------------------------------------
# GRÁFICO 1: Calcular Distâncias
# -----------------------------------------------------
ax1 = axes[0]

for i in range(3):
    indices = y_amostra == i
    ax1.scatter(X_amostra[indices, 0], X_amostra[indices, 1], 
                c=cores[i], s=150, alpha=0.7, label=labels[i].capitalize(),
                edgecolors='k', linewidth=1.5, zorder=3)
    
ax1.scatter(novo_ponto_viz[0], novo_ponto_viz[1], c='gold', s=400, marker='*', 
            label='Flor Desconhecida', edgecolors='k', linewidth=1.5, zorder=10)

for i, ponto in enumerate(X_amostra):
    ax1.plot([novo_ponto_viz[0], ponto[0]], [novo_ponto_viz[1], ponto[1]], 
             'gray', alpha=0.3, linewidth=1.5, linestyle='-', zorder=1)

ax1.set_xlabel("Comprimento da Pétala (cm)", fontsize=12)
ax1.set_ylabel("Largura da Pétala (cm)", fontsize=12)
ax1.set_title("Passo 1: Calcular Distâncias")
ax1.legend(loc='upper left', fontsize=10, framealpha=0.95)
ax1.set_xlim(0, 7)
ax1.set_ylim(0, 3)

# -----------------------------------------------------
# GRÁFICO 2: Encontrar K Vizinhos
# -----------------------------------------------------
ax2 = axes[1]

for i in range(3):
    indices = y_amostra == i
    ax2.scatter(X_amostra[indices, 0], X_amostra[indices, 1], 
                c=cores[i], s=80, alpha=0.4,
                edgecolors='gray', linewidth=0.5, zorder=1)

# Destacando os k vizinhos mais próximos
vizinhos_pts = X_amostra[vizinhos_viz]
vizinhos_cls = y_amostra[vizinhos_viz]
cores_viz = [cores[int(cls)] for cls in vizinhos_cls]

ax2.scatter(vizinhos_pts[:, 0], vizinhos_pts[:, 1],
            c=cores_viz, s=300, alpha=0.95,
            edgecolors='black', linewidth=2.5, zorder=5)
    
ax2.scatter(novo_ponto_viz[0], novo_ponto_viz[1], c='gold', s=400, marker='*',
            edgecolors='k', linewidth=2.5, zorder=10)

ax2.set_xlabel("Comprimento da Pétala (cm)", fontsize=12)
ax2.set_title("Passo 2: Encontrar os K Vizinhos Mais Próximos")
ax2.set_xlim(0, 7)
ax2.set_ylim(0, 3)

# -----------------------------------------------------
# GRÁFICO 3: Prever a Classe
# -----------------------------------------------------
ax3 = axes[2]

for i in range(3):
    indices = y_amostra == i
    ax3.scatter(X_amostra[indices, 0], X_amostra[indices, 1], 
                c=cores[i], s=60, alpha=0.3,
                edgecolors='gray', linewidth=0.3, zorder=1)
    
ax3.scatter(vizinhos_pts[:, 0], vizinhos_pts[:, 1],
            c=cores_viz, s=180, alpha=0.5,
            edgecolors='gray', linewidth=1.5, zorder=3)

cor_prev = cores[int(classe_viz)]
ax3.scatter(novo_ponto_viz[0], novo_ponto_viz[1], c=cor_prev, s=500, marker='*',
            edgecolors='k', linewidth=3, zorder=10, alpha=0.95, 
            label=f'Classe: {labels[int(classe_viz)].capitalize()}')

from matplotlib.patches import Circle
circulo = Circle(novo_ponto_viz, dist_viz[vizinhos_viz[-1]],
                 fill=False, edgecolor=cor_prev, linewidth=3, linestyle='--', alpha=0.7, zorder=2)
ax3.add_patch(circulo)

ax3.set_xlabel("Comprimento da Pétala (cm)", fontsize=12)
ax3.set_title("Passo 3: Prever a Classe")
ax3.set_xlim(0, 7)
ax3.set_ylim(0, 3)
ax3.legend(loc='upper left', fontsize=10, framealpha=0.95)

plt.tight_layout()
plt.show()

<a id='formalizacao'></a>
## **Formalização Matemática**

Agora que você entendeu intuitivamente como o KNN funciona, vamos formalizar o algoritmo matematicamente.

### **Entrada do Algoritmo**

O KNN recebe como entrada:

- **Conjunto de treinamento:** $D = \{(x_1, y_1), (x_2, y_2), ..., (x_n, y_n)\}$
  
  Onde:
  - $x_i$: vetor de características (no nosso exemplo: comprimento e largura da pétala)
  - $y_i$: rótulo/classe (no nosso exemplo: Setosa, Versicolor, Virginica)
  - Corresponde à variável `X_train` no código

- **Hiperparâmetro K:** Número de vizinhos a serem considerados (variável `k` no código)

- **Ponto para previsão:** $x_0$ (a nova observação que queremos classificar — variável `novo_ponto` no código)

### **Processo do Algoritmo**

**1. Calcular Distâncias**

Para cada ponto $x_i$ no conjunto de treinamento, calculamos a distância até $x_0$. A métrica mais comum é a **distância euclidiana**:

$$d(x_0, x_i) = \sqrt{\sum^n_{j=1}(x_{0j} - x_{ij})^2}$$

Essa é simplesmente a distância em linha reta entre dois pontos em um espaço de $n$ dimensões (onde $n$ é o número de features). No código, implementamos isso através da função `euclidean_distance(X, point)` que retorna um array com todas as distâncias.

**Nota:** Outras métricas de distância podem ser usadas dependendo do problema:
- **Distância de Manhattan:** $d(x_0, x_i) = \sum^n_{j=1}|x_{0j} - x_{ij}|$
- **Distância de Minkowski:** Generalização das anteriores
- **Distância de Hamming:** Para dados categóricos

**2. Ordenar e Identificar os Vizinhos**

Após calcular todas as distâncias:
- Ordenamos as distâncias em ordem crescente
- Selecionamos o conjunto $N$ que contém os $K$ pontos com as $K$ menores distâncias
- Esse conjunto $N$ forma a "vizinhança" do ponto $x_0$

No código, usamos `np.argsort(distancias)[:k]` para obter os índices dos K vizinhos mais próximos (função `find_knn`).

**Importante sobre a escolha de K:**
- K pequeno (ex: K=1): Modelo sensível a ruído, fronteiras complexas
- K grande (ex: K=50): Modelo mais suave, mas pode ignorar padrões locais
- K ótimo: Geralmente encontrado por validação cruzada (veremos na seção de técnicas avançadas)

### **Saída: Previsão**

A previsão $\hat{y}$ para classificação é simplesmente a **classe mais frequente** (moda) entre os rótulos dos $K$ vizinhos no conjunto $N$.

Formalmente:

$$\hat{y} = \arg\max_v \sum_{(x_i, y_i) \in N} I(y_i = v)$$

Onde:
- $\arg\max_v$: encontra o valor $v$ que maximiza a expressão
- $I(y_i = v)$: função indicadora que vale 1 se $y_i = v$, e 0 caso contrário
- Traduzindo: "conte os votos para cada classe $v$ e escolha a que tiver mais"

### **Pseudocódigo**

```
Algoritmo KNN(D, K, x₀):
    1. Para cada (xᵢ, yᵢ) em D:
        a. Calcular d(x₀, xᵢ)
    
    2. Ordenar as distâncias em ordem crescente
    
    3. Selecionar os K vizinhos mais próximos → conjunto N
    
    4. Contar votos: para cada classe v, contar quantos vizinhos têm yᵢ = v
    
    5. Retornar a classe com mais votos
```

<a id='from-scratch'></a>
## **Implementação do Zero**

Agora que você entende a matemática, vamos implementar o KNN completamente do zero, usando apenas NumPy!

### **Desafio: Implemente você mesmo!**

Tente completar a classe KNN abaixo. Use o que aprendemos nos passos anteriores como referência:

In [ ]:
class KNN:
    def __init__(self, k=3):
        """
        Inicializa o classificador KNN

        Parâmetros:
        k (int): Número de vizinhos a considerar (padrão é 3)
        """

        pass

    def fit (self, X_train, y_train):
        """
        Armazena os dados de treinamento

        Parâmetros:
        X_train (array-like): Dados de treinamento (features)
        y_train (array-like): Labels de treinamento (classes)
        """

        pass

    def _euclidean_distance(self, x1, x2):
        """
        Calcula a distância Euclidiana entre dois pontos

        Parâmetros:
        x1, x2 (array-like): Pontos para calcular a distância

        Retorna:
        float: Distância Euclidiana
        """

        pass

    def predict(self, X_test):
        """
        Faz previsões para os dados de teste

        Parâmetros:
        X_test (array-like): Dados de teste (features)

        Retorna:
        array: Previsões das classes
        """

        pass

    def _predict_single(self, x):
        """
        Faz a previsão para um único ponto de dados

        Parâmetros:
        x (array-like): Ponto de dados

        Retorna:
        int: Classe prevista
        """

        pass

    def score(self, X_test, y_test):
        """
        Calcula a acurácia do classificador

        Parâmetros:
        X_test (array-like): Dados de teste (features)
        y_test (array-like): Labels verdadeiros (classes)

        Retorna:
        float: Acurácia do classificador
        """

        pass


In [ ]:
# Dados para Teste (dataset Iris)
# Carregar o dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividir em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

<a id='sklearn'></a>
## **Implementação com Scikit-Learn**

Na prática, usamos bibliotecas otimizadas como o Scikit-Learn. Vamos ver como é simples implementar o mesmo algoritmo:

### **Implementação Básica**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)  # Modelo com K=7

knn.fit(X_train, y_train)  # Treinando o modelo

predictions = knn.predict(X_test)  # Fazendo previsões

# Avaliando o modelo
accuracy = accuracy_score(y_test, predictions)
print(f"Acurácia: {accuracy*100:.2f}%")

### **Técnicas Avançadas**

Agora vamos explorar algumas técnicas importantes para melhorar o desempenho do KNN:

In [ ]:
# ===================================================================
# TÉCNICA 1: Cross-Validation (Validação Cruzada)
# ===================================================================

from sklearn.model_selection import cross_val_score

print("Cross-Validation: encontrando o melhor K\n")
print("Objetivo:")
print("  Dividir dados em 'folds' (partes)")
print("  Treinar em alguns folds e testar em outros")
print("  Repetir várias vezes e calcular média da acurácia\n")

k_range = range(1, 31)
k_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')
    k_scores.append(scores.mean())

best_k = k_range[np.argmax(k_scores)]
best_score = max(k_scores)

print("=" * 70)
print(f"MELHOR K: {best_k}")
print(f"Acurácia média (5-fold CV): {best_score*100:.2f}%")
print("=" * 70)

In [ ]:
# ===================================================================
# TÉCNICA 2: Normalização (Crucial para KNN!)
# ===================================================================

print("Normalização: por que é importante para KNN?\n")
print("Problema:")
print("  • KNN usa distância Euclidiana")
print("  • Features com escalas diferentes dominam o cálculo")
print("  • Ex: feature de 0-1000 vs feature de 0-1\n")

print("Solução:")
print("  • Normalizar: média 0, desvio padrão 1")
print("  • Coloca todas features na mesma escala\n")

# Criando dados com escalas diferentes
from sklearn.datasets import make_classification
X_test_scale, y_test_scale = make_classification(n_samples=200, n_features=2, n_informative=2, 
                                                   n_redundant=0, random_state=42)
X_test_scale[:, 1] = X_test_scale[:, 1] * 100  # Segunda feature com escala 100x maior

X_train_us, X_test_us, y_train_us, y_test_us = train_test_split(
    X_test_scale, y_test_scale, test_size=0.2, random_state=42
)

print("Sem Normalização:")
knn_without = KNeighborsClassifier(n_neighbors=5)
knn_without.fit(X_train_us, y_train_us)
acc_without = knn_without.score(X_test_us, y_test_us)
print(f"  Acurácia: {acc_without*100:.2f}%\n")

print("Com Normalização:")
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train_us)
X_test_norm = scaler.transform(X_test_us)

knn_with = KNeighborsClassifier(n_neighbors=5)
knn_with.fit(X_train_norm, y_train_us)
acc_with = knn_with.score(X_test_norm, y_test_us)
print(f"  Acurácia: {acc_with*100:.2f}%\n")

print(f"Melhoria de {((acc_with - acc_without) / acc_without * 100):.1f}%")
print("\nSEMPRE normalize seus dados ao usar KNN!")

In [ ]:
# ===================================================================
# TÉCNICA 3: Pipeline (Fluxo Completo)
# ===================================================================

print("Pipeline: encadeando normalização + modelo\n")
print("Vantagens:")
print("  Facilita deployment")
print("  Previne data leakage")
print("  Garante mesmo pré-processamento em treino e teste\n")

# Criando pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),           # Etapa 1: Normalização
    ('knn', KNeighborsClassifier(n_neighbors=5))  # Etapa 2: Modelo
])

# Usando o pipeline (fit + transform + predict em uma linha!)
pipeline.fit(X_train, y_train)
predictions_pipeline = pipeline.predict(X_test)

accuracy_pipeline = accuracy_score(y_test, predictions_pipeline)
print("=" * 70)
print(f"Acurácia com Pipeline: {accuracy_pipeline*100:.2f}%")
print("=" * 70)

print("\nPipeline aplica automaticamente as transformações na ordem correta!")

<a id='vantagens-desvantagens'></a>
## **Vantagens e Desvantagens do KNN**

Agora que você conhece o KNN profundamente, vamos resumir seus pontos fortes e fracos:

### **✅ Vantagens**

1. **Algoritmo intuitivo e simples:** Fácil de entender e explicar
2. **Não há fase de treinamento:** Apenas armazena os dados
3. **Flexível:** Funciona bem com dados não-lineares
4. **Naturalmente multiclasse:** Sem necessidade de adaptações
5. **Atualização incremental:** Novos dados podem ser adicionados facilmente

### **❌ Desvantagens**

1. **Lento na previsão:** Precisa calcular distância para todos os pontos de treino
2. **Sensível à escala:** Features com escalas diferentes dominam o cálculo
3. **Maldição da dimensionalidade:** Performance cai drasticamente com muitas features
4. **Sensível a dados esparsos:** Ruim quando a maioria dos valores é zero
5. **Escolha do K:** Requer tuning, não há valor universalmente ótimo
6. **Alto uso de memória:** Precisa armazenar todo o conjunto de treino

### **Quando usar KNN?**

✅ **Bom para:**
- Datasets pequenos a médios
- Poucas features (<20)
- Fronteiras de decisão irregulares
- Baseline rápido

❌ **Evitar quando:**
- Datasets muito grandes (milhões de exemplos)
- Muitas features (centenas/milhares)
- Dados esparsos (texto, por exemplo)
- Necessidade de previsões muito rápidas

---

<-- [**Anterior: Métricas de Classificação**](../02_Fundamentos_ML/02_metricas_classificacao.ipynb) | [**Próximo: Naive Bayes**](02_naive_bayes.ipynb) -->